In [1]:
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from scipy.integrate import odeint
import scipy
import numpy as np
from scipy.integrate import solve_ivp

In [2]:
class DifferentialEquation:
    def __init__(self, initial_values):
        self.initial_values = initial_values

    def step(self, state, t):
        pass

    def solve(self, dt, time, solver):
        return solver.solve(self, dt, time)

In [3]:
class Solver(ABC):
    @abstractmethod
    def solve(self, equation, dt, time):
        pass

In [4]:
class ErrorFinder:
    def __init__(self, error_finder):
        self.error_finder = error_finder

    def error_measure(self):
        pass

In [5]:
class NumericalEquation(DifferentialEquation):
    def __init__(self):
        pass
    def step(self, state, t):
        pass

In [6]:
class AnalyticalEquation(DifferentialEquation):
    def __init__(self):
        pass
    def equation(self,t):
        pass

In [7]:
class NumericalSolver(Solver):
    def __init__(self):
        pass
    def solve(self):
        pass

In [8]:
class AnalyticalSolver(Solver):
    def __init__(self):
        pass
    def solve(self):
        pass

In [7]:
class EulerSolver(NumericalSolver):
    def __init__(self):
        pass

    def solve(self, equation,initial_values, dt, time):
        t = np.arange(0, time, dt)
        states = np.zeros((len(t), len(initial_values)))
        states[0] = initial_values

        for i in range(len(t) - 1):
            state = states[i]
            state_dt = equation.step(state, t[i])
            state_next = state + np.array(state_dt) * dt
            states[i + 1] = state_next

        return states

In [17]:
class AdvancedErrorFinder(ErrorFinder):
    def __init__(self,app_matrix,exact_matrix):
        self.app_matrix = app_matrix
        self.exact_matrix = exact_matrix

    def relative_error(self):
        app = self.app_matrix
        exact = self.exact_matrix
        error = exact - app
        return np.mean(np.abs(error / exact), axis=0)

    def kolmogorov_smirnov_test(self):
        app = self.app_matrix
        exact = self.exact_matrix
        error = exact - app
        mean_error = np.mean(error, axis=0)
        std_error = np.std(error, axis=0)
        ks_statistic, p_value = scipy.stats.kstest((error - mean_error) / std_error, 'norm')
        return ks_statistic, p_value

    def akaike_information_criterion(self, num_params):
        app = self.app_matrix
        exact = self.exact_matrix
        error = exact - app
        num_time_steps = app.shape[0]
        mse = np.sum(error**2) / (num_time_steps * app.shape[1])
        aic = num_time_steps * app.shape[1] * np.log(mse) + 2 * num_params
        return aic

In [16]:
class BasicErrorFinder(ErrorFinder):

    def __init__(self,app_matrix,exact_matrix):
        self.app_matrix = app_matrix
        self.exact_matrix = exact_matrix

    def mean_residual_error(self):
        app = self.app_matrix
        exact = self.exact_matrix
        error = exact - app
        return np.mean(error,axis=0)

    def mean_squared_error(self):
        app = self.app_matrix
        exact = self.exact_matrix
        error = exact - app
        return np.mean(error**2,axis=0)

    def root_mean_squared_error(self):
        app = self.app_matrix
        exact = self.exact_matrix
        error = exact - app
        return np.sqrt(np.mean(error**2, axis=0))

    def max_absolute_error(self):
        app = self.app_matrix
        exact = self.exact_matrix
        error = np.abs(exact - app)
        return np.max(error, axis=0)

In [8]:
class OdeintSolver(NumericalSolver):
    def __init__(self):
        pass

    def solve(self, equation,initial_values, dt,time):
        t = np.linspace(0, time, int(time / dt))
        states = odeint(equation.step, initial_values, t)
        return states

In [9]:
class LorenzNumeric(NumericalEquation):
    state_variables = ['x','y','z']
    def __init__(self, s, r, b,dimension=3):
        self.s = s
        self.r = r
        self.b = b
        self.dimension = dimension

    def step(self, state, t):
        x, y, z = state
        x_dt = self.s * (y - x)
        y_dt = self.r * x - y - x * z
        z_dt = x * y - self.b * z
        return [x_dt, y_dt, z_dt]

In [10]:
class LotkaVolterraNumeric(NumericalEquation):
    def __init__(self, alpha, beta, gamma, delta,dimension=2):
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.delta = delta
        self.dimension = dimension
    def step(self, state, t):
        x, y = state
        x_dt = self.alpha * x - self.beta * x * y
        y_dt = self.delta * x * y - self.gamma * y
        return [x_dt, y_dt]

In [11]:
class Plotter:
    def __init__(self, equation_solver):
        self.equation_solver = equation_solver

    def plot3d(self, equation,initial_values, color, dt, end, name='3D Plot'):
        xyz = self.equation_solver.solve(equation,initial_values, dt, end)
        fig = plt.figure(figsize=(12, 9))
        num_axes = xyz.shape[1]
        axs = fig.subplots(nrows=1, ncols=num_axes, subplot_kw={'projection': '3d'})

        for i in range(num_axes):
            ax = axs[i]
            ax.plot(xyz[:, i], xyz[:, (i+1)%num_axes], xyz[:, (i+2)%num_axes], color=color, alpha=0.7, linewidth=0.7)
            ax.set_xlabel(f"os {i}")
            ax.set_ylabel(f"os {(i+1)%num_axes}")
            ax.set_zlabel(f"os {(i+2)%num_axes}")
            ax.set_title(name)

        plt.show()

    def plot_combinations(self, equation,initial_values, dt, time, color, names=None):
        xyz = self.equation_solver.solve(equation,initial_values, dt, time)
        num_plots = xyz.shape[1] * (xyz.shape[1] - 1) // 2

        if names is None:
            names = [f"Variable {i+1}" for i in range(xyz.shape[1])]

        fig, axs = plt.subplots(1, num_plots, figsize=(18, 5))

        axs = np.ravel(axs) # convert axs to a 1D numpy array

        plot_index = 0
        for i in range(xyz.shape[1]):
            for j in range(i+1, xyz.shape[1]):
                ax = axs[plot_index]
                ax.plot(xyz[:, i], xyz[:, j], color=color, markersize=1)
                ax.set_xlabel(names[i])
                ax.set_ylabel(names[j])
                plot_index += 1

        plt.show()